# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g_api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load CSV file
csv_file = "/Users/ashleypatricia/Documents/GitHub/python-api-challenge/WeatherPy/city data.csv"

# Assign dataframe 
weather_df = pd.read_csv(csv_file)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,dikson,73.5069,80.5464,19.18,93,100,21.16,RU,1619408280
1,pevek,69.7008,170.3133,25.61,84,99,2.33,RU,1619408281
2,kaitangata,-46.2817,169.8464,48.00,68,100,8.01,NZ,1619408697
3,bintulu,3.1667,113.0333,87.80,70,40,2.30,MY,1619408698
4,souillac,-20.5167,57.5167,77.00,88,75,14.97,MU,1619408699
...,...,...,...,...,...,...,...,...,...
565,berlevag,70.8578,29.0864,32.00,93,40,11.50,NO,1619409064
566,belaya gora,68.5333,146.4167,27.95,74,88,11.43,RU,1619409065
567,hualmay,-11.0964,-77.6139,65.80,89,17,8.03,PE,1619409065
568,xinmin,41.9908,122.8253,75.99,17,0,13.42,CN,1619409066


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations 
locations = weather_df[["Lat", "Lng"]]

# Use humidity as weight 
humidity = weather_df["Humidity"].astype(float)

# Plot heatmap
fig = gmaps.figure()

# Add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer to plot 
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#### Narrow down to cities that only fit 3 ideal weather conditions

#A max temperature lower than 80 degrees but higher than 70
temp_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70)] 
temp_df

# Wind speed less than 10 mph.
ws_df = temp_df.loc[temp_df["Wind Speed"] < 10]
ws_df

# Zero cloudiness
cloudiness_df = ws_df.loc[ws_df["Cloudiness"] == 0]
cloudiness_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
140,kibala,9.1104,18.3463,75.20,78,0,5.61,TD,1619408783
224,umm lajj,25.0213,37.2685,79.95,53,0,5.48,SA,1619408450
239,beloha,-25.1667,45.0500,70.66,74,0,5.61,MG,1619408261
320,djougou,9.7085,1.6660,78.80,78,0,4.61,BJ,1619408527
372,rock sound,24.9000,-76.2000,78.19,83,0,8.70,BS,1619408579
394,atar,20.5169,-13.0499,75.58,22,0,6.76,MR,1619408593
420,ndioum,16.5129,-14.6471,73.40,60,0,4.61,SN,1619408618
465,ulladulla,-35.3500,150.4667,71.60,37,0,4.61,AU,1619408987
467,pauri,30.1500,78.7833,75.54,19,0,1.30,IN,1619408988
469,iquitos,-3.7481,-73.2472,78.80,89,0,6.91,PE,1619408989


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Clean up dataframe and exclude columns with weather condition data
hotel_df = pd.DataFrame(cloudiness_df, columns=["City", "Country", "Lat", "Lng"])

# Add new column called "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Hotel Name
140,kibala,TD,9.1104,18.3463,
224,umm lajj,SA,25.0213,37.2685,
239,beloha,MG,-25.1667,45.0500,
320,djougou,BJ,9.7085,1.6660,
372,rock sound,BS,24.9000,-76.2000,
394,atar,MR,20.5169,-13.0499,
420,ndioum,SN,16.5129,-14.6471,
465,ulladulla,AU,-35.3500,150.4667,
467,pauri,IN,30.1500,78.7833,
469,iquitos,PE,-3.7481,-73.2472,


In [13]:
# Set up empty list to hold hotel names 
hotelnames = []

# Create for loop
for index, row in hotel_df.iterrows(): 
    lat = row["Lat"]
    lng = row["Lng"]
    name = row["City"]
    county = row["Country"]

    # Set parameters to search for hotels with 5000 meters.
    target_coordinates = f"{lat}, {lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    params = {
        "location": target_coordinates, 
        "keyword": target_search, 
        "radius": target_radius,
        "type": target_type, 
        "key": g_key
        }
    # Search Google Places API for each city's coordinates
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response = response.json()

    # Append names of hotels for each city 
    try: 
        hotelnames.append(response["results"][0]['name'])

    # If there is not a result, print "None Found"
    except (KeyError, IndexError): 
        hotelnames.append("None Found")

# Store the first hotel result into the data frame for each city 
hotel_df["Hotel Name"] = hotelnames
hotel_df

,City,Country,Lat,Lng,Hotel Name
140,kibala,TD,9.1104,18.3463,hotel safari
224,umm lajj,SA,25.0213,37.2685,HP Red Sea Hotel
239,beloha,MG,-25.1667,45.0500,None Found
320,djougou,BJ,9.7085,1.6660,Motel Du Lac
372,rock sound,BS,24.9000,-76.2000,Pink Sandy Beaches Villas
394,atar,MR,20.5169,-13.0499,Bab Sahara
420,ndioum,SN,16.5129,-14.6471,Mbathior Hôtel
465,ulladulla,AU,-35.3500,150.4667,Mollymook Shores Motel and Conference Centre
467,pauri,IN,30.1500,78.7833,Hotel Sun n Snow
469,iquitos,PE,-3.7481,-73.2472,DoubleTree by Hilton Iquitos


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))